In [1]:
import numpy as np

# Problema 1

Numarul de operatii care se efectueaza in eliminarea Gaussiana este:

$$
\sum^{n-1}_{k=1}(n-k)(n-k+1)2
$$

In [2]:
def gauss_complexity(n):
    c = 0
    for k in range(1, n):
        c += (n-k)*(n-k+1)*2
    return c
gauss_complexity(4)

40

Asadar complexitatea metodei lui Gauss pentru $n=4$ va fi $40$.

# Problema 2

In [3]:
def gen_A(n):
    assert n > 0 and n % 2 == 0

    sd = np.ones(n)
    sd[[n//2-1, n//2]] = 0
    
    A = (
        + np.diag(np.ones(n)*3, 0)
        + np.diag(np.ones(n-1)*-1, 1)
        + np.diag(np.ones(n-1)*-1, -1)
        + np.fliplr(np.diag(sd*(1/2)))
    )
    return A

gen_A(10)

array([[ 3. , -1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0.5],
       [-1. ,  3. , -1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0.5,  0. ],
       [ 0. , -1. ,  3. , -1. ,  0. ,  0. ,  0. ,  0.5,  0. ,  0. ],
       [ 0. ,  0. , -1. ,  3. , -1. ,  0. ,  0.5,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. , -1. ,  3. , -1. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. , -1. ,  3. , -1. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0.5,  0. , -1. ,  3. , -1. ,  0. ,  0. ],
       [ 0. ,  0. ,  0.5,  0. ,  0. ,  0. , -1. ,  3. , -1. ,  0. ],
       [ 0. ,  0.5,  0. ,  0. ,  0. ,  0. ,  0. , -1. ,  3. , -1. ],
       [ 0.5,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. , -1. ,  3. ]])

In [4]:
def gen_b(n):
    b = np.ones(n)*1.5
    b[[0, -1]] = 2.5
    b[[n//2-1, n//2]] = 1
    b = b.reshape(-1, 1)
    return b

gen_b(10)

array([[2.5],
       [1.5],
       [1.5],
       [1.5],
       [1. ],
       [1. ],
       [1.5],
       [1.5],
       [1.5],
       [2.5]])

In [5]:
def lup_decomp(A):
    assert len(A.shape) == 2
    assert A.shape[0] == A.shape[1]
    n = A.shape[0]
    A = A.copy()
    
    P = np.eye(n)
    
    for k in range(n-1):
        i = k + np.argmax(np.abs(A[k:, k]))
        if i != k:
            A[[i, k]] = A[[k, i]]
            P[[i, k]] = P[[k, i]]
        
        lin = np.s_[k+1:n]
        
        A[lin, k] /= A[k, k]
        A[lin, lin] -= np.matmul(A[lin, [k]], A[[k], lin])
    
    L = np.tril(A)
    np.fill_diagonal(L, 1)
    
    U = np.triu(A)

    return L, U, P


def lup_solve(A, b):
    assert len(A.shape) == 2
    assert A.shape[0] == A.shape[1]
    n = A.shape[0]
    
    L, U, P = lup_decomp(A)
    
    Pb = np.matmul(P, b)
    
    y = np.zeros(n)
    for i in range(n):
        y[i] = (Pb[i, 0] - np.sum(L[i, :i] * y[:i])) / L[i, i]
    y = y.reshape(-1, 1)
    
    x = np.zeros(n)
    for i in range(n-1, -1, -1):
        x[i] = (y[i, 0] - np.sum(U[i, i:] * x[i:])) / U[i, i]
    x = x.reshape(-1, 1)

    return x

In [6]:
def cholesky_decomp(A):
    assert len(A.shape) == 2
    assert A.shape[0] == A.shape[1]
    n = A.shape[0]
    
    # check A is positively defined just in case
    for i in range(n):
        a = A[:i+1, :i+1]
        assert np.linalg.det(a) > 0

    L = np.zeros_like(A)
    for k in range(n):
        L[k, k] = np.sqrt(A[k, k] - np.sum(L[k, :k]**2))
        for i in range(k+1, n):
            L[i, k] = (A[i, k] - np.sum(L[i, :k]*L[k, :k])) / L[k, k]
    return L


def cholesky_solve(A, b):
    assert len(A.shape) == 2
    assert A.shape[0] == A.shape[1]
    n = A.shape[0]
    
    L = cholesky_decomp(A)

    y = np.zeros(n)
    for i in range(n):
        y[i] = (b[i, 0] - np.sum(L[i, :i] * y[:i])) / L[i, i]
    y = y.reshape(-1, 1)
    
    Lt = np.transpose(L)
    
    x = np.zeros(n)
    for i in range(n-1, -1, -1):
        x[i] = (y[i, 0] - np.sum(Lt[i, i:] * x[i:])) / Lt[i, i]
    x = x.reshape(-1, 1)

    return x

## (a)

In [7]:
n = 10
A = gen_A(n)
print(f"{A=}")

A=array([[ 3. , -1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0.5],
       [-1. ,  3. , -1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0.5,  0. ],
       [ 0. , -1. ,  3. , -1. ,  0. ,  0. ,  0. ,  0.5,  0. ,  0. ],
       [ 0. ,  0. , -1. ,  3. , -1. ,  0. ,  0.5,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. , -1. ,  3. , -1. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. , -1. ,  3. , -1. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0.5,  0. , -1. ,  3. , -1. ,  0. ,  0. ],
       [ 0. ,  0. ,  0.5,  0. ,  0. ,  0. , -1. ,  3. , -1. ,  0. ],
       [ 0. ,  0.5,  0. ,  0. ,  0. ,  0. ,  0. , -1. ,  3. , -1. ],
       [ 0.5,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. , -1. ,  3. ]])


Descompunerea LUP

In [8]:
L, U, P = lup_decomp(A)
print(f"{L=}")
print(f"{U=}")
print(f"{P=}")

L=array([[ 1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [-0.33333333,  1.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        , -0.375     ,  1.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        , -0.38095238,  1.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , -0.38181818,  1.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        , -0.38194444,
         1.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.19090909,  0.07291667,
        -0.35411141,  1.        ,  0.      

Descompunerea Cholesky

In [9]:
L = cholesky_decomp(A)
print(f"{L=}")

L=array([[ 1.73205081,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [-0.57735027,  1.63299316,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        , -0.61237244,  1.62018517,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        , -0.6172134 ,  1.61834719,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , -0.61791438,  1.61807967,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        , -0.61801654,
         1.61804065,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.30895719,  0.11798498,
        -0.57296665,  1.60072928,  0.      

## (b)

In [10]:
n = 100
A = gen_A(n)
b = gen_b(n)

Rezolvare prin descompunerea LUP

In [11]:
print(lup_solve(A, b))

[[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]]


Rezolvare prin descompunerea Cholesky

In [12]:
print(cholesky_solve(A, b))

[[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]]
